# Lab1.2: Databases and SQL

## Outline

* Introduction to SQL
* Practice writing basic SQL queries

## Why learn RDBMS/SQL?

RDBMS = Relational Database Management System

SQL = Structured Query Language

SQL is everywhere. It will often be the tool between you and the data you want. The majority of businesses store their data in a RDBMS and use SQL or SQL-like tools to access it. The combination is exceptionally good at efficiently storing complicated data sets while allowing for efficient information retrieval.

Data is stored in a SQL server, usually connected to remotely. For the exercises and examples, we'll be locally hosting our database. This is somewhat atypical of what you'll experience in the field.

Example:
Web front end <----> SQL Server <----> Data Queries


## Data Persistence

The most common form of data persistence is the flat file. You open up a text
editor, enter some information, and save the file to disk. You've just persisted
some data and can transfer that file wherever you wish. For most data, that's
enough, but for many applications, there are additional constraints other than
persistence and portability.

For example,

  * Ease of sharing between programming languages
  * Reliability guarantees 
  * Data integrity guarantees (e.g. no duplication)
  * Ability to query data efficiently
  * Ability to model relations in data

The first constraint is easily met by data formats such as JSON, CSV, or XML.
All mainstream programming languages have libraries to convert these formats to
native data structures and back to the interchange format. For example, Python
has the `json` module that converts between JSON strings and Python dicts.

However, these widely used data formats cannot meet the other constraints
listed above (and this is by no means an exhaustive list of desirable
constraints on data persistence).

## Relational Database Management Systems (RDBMS)

Relational databases such as PosgreSQL, MySQL, and SQLite were built for just
those purposes. They provide the ability to model relations in data and query
the data and their relations efficiently. They also provide a bevy of guarantees
to maintain data consistency and integrity.

*Note: The sorts of guarantees that these databases provide is beyond
the scope of this document (and this class), but feel free to reference the
resources at the bottom of this document if you're interested in learning more
about the underlying theory of relational databases.*

*Second Note: there are non-relational databases (NoSQL/MongoDB, etc.) that store data with different constraints and mechanisms for relationships between data subsets. We'll touch on these later in the course.*

## The RDBMS Data Model

To proceed with this lab, first install the `ipython-sql` extension:

In [1]:
!pip install ipython-sql

Load it using the `%load_ext` magic, which will impart `sql` magic to Jupyter (yes, it's actually called "magic").

In [2]:
%load_ext sql

/Users/alessandro/anaconda/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/Users/alessandro/anaconda/lib/python3.5/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


<small>You may get a `ShimWarning` warning here. Don't worry about it. It's a minor bug that should be fixed in a later release of `ipython-sql` but shouldn't effect your work.</small>

Connect to a dummy `sqlite` database as such:

In [3]:
%sql sqlite://

'Connected: None@None'

Relational databases have a **schema** that defines the structure of the data. It's set when the database is created and is difficult to change later.

Each database is composed of a number of user-defined **tables**, each with
**columns** and **rows**. Each column is of a certain **data type** such as
integer, string, or date. Each row is an entry in the table with data for each
column of that table.

Here's an example of a database table creation specifying a `customers` table with various
fields and their data types:

In [4]:
%%sql
DROP TABLE IF EXISTS customers;  -- Included so we can run this multiple times without error.

CREATE TABLE customers (
    id INTEGER PRIMARY KEY
,   name VARCHAR(50)
,   age INTEGER
,   city VARCHAR(255)
,   state VARCHAR(2));

Done.
Done.


[]

The data types available to you vary from system to system. These above will work in either PostgreSQL or SQLite. `VARCHAR` is a string data type.

A **primary key** is a column in a table that uniquely identifies that entry. No
two rows in the same table can share a value for a column specified as primary
key. The primary key column is most often `id`.

The table is currently empty:

In [5]:
%sql SELECT * FROM customers

Done.


id,name,age,city,state


We can `INSERT` data into it as follows:

In [6]:
%%sql
INSERT INTO customers VALUES 
    (1, 'john', 25, 'San Francisco', 'CA'),
    (2, 'becky', 30, 'NYC', 'NY'),
    (3, 'sarah', 20, 'Denver', 'CO');

3 rows affected.


[]

Now with data:

In [7]:
%sql SELECT * FROM customers

Done.


id,name,age,city,state
1,john,25,San Francisco,CA
2,becky,30,NYC,NY
3,sarah,20,Denver,CO


## Modeling Relations in RDBMS

Part of the power of relational databases are their ability to model relations
in data. The way they do so is through the use of **foreign keys**.
A foreign key is a column that references some other entry in the database.
That foreign entry could be in the same table or in some other table. Foreign
keys are how relations are modeled in relational databases.

For example, let's say there is another table that contains data for each visit
to our website. Each time a customer visits the site, a row is created and inserted
into the `visits` table. We'd like to maintain some data that indicates which
visit is associated with which customer (so that we could later, for example, find
the customers who visited our site the most). Each visit then will have a `customer_id`
that can connect it to a customer in the `customers` table.

Here's the definition of the `visits` table:

In [8]:
%%sql
DROP TABLE IF EXISTS visits;  -- Included so we can run this multiple times without error.

CREATE TABLE visits (
    created_at TIMESTAMP
,   customer_id INTEGER REFERENCES customers(id) 
,   PRIMARY KEY(created_at, customer_id));

INSERT INTO visits VALUES 
    ('2016-06-20', 1),
    ('2016-07-30', 1),
    ('2016-06-20', 3),
    ('2016-04-09', 1),
    ('2016-03-09', 2);

Done.
Done.
5 rows affected.


[]

Here's an example of what this visits table looks like:

In [9]:
%sql SELECT * FROM visits

Done.


created_at,customer_id
2016-06-20,1
2016-07-30,1
2016-06-20,3
2016-04-09,1
2016-03-09,2


Here we specify not only that the `visits` table has a column called `customer_id`,
but that the column references the `id` column in the `customers` table. PostgreSQL
will treat this as a constraint and ensure that new visits have a `customer_id`
value that references an actual customer in the database.

### Types of Relationships

#### One-to-one

For a one-to-one relationship, place the foreign key on either side of the
relationship.

**Example: Customers and Licenses**
Licenses is a table that holds the _most recent_ driver's license number for a customer,
because customers can only have 1 driver's license at time, it will be a one-to-one relationship.

In [10]:
%%sql
DROP TABLE IF EXISTS licenses;  -- Included so we can run this multiple times without error.

CREATE TABLE licenses (
  state VARCHAR(2)
, number VARCHAR(20)
, uploaded_at TIMESTAMP
, customer_id INTEGER REFERENCES customers(id)
, PRIMARY KEY(state, number));

INSERT INTO licenses VALUES 
    ('CO', 'DL19480284', '2013-04-18', 3),
    ('CA', 'DL19852984', '2014-05-12', 1);

SELECT * FROM licenses;

Done.
Done.
2 rows affected.
Done.


state,number,uploaded_at,customer_id
CO,DL19480284,2013-04-18,3
CA,DL19852984,2014-05-12,1


To find the license for a customer:

In [11]:
%%sql
SELECT *
FROM licenses
WHERE customer_id=1

Done.


state,number,uploaded_at,customer_id
CA,DL19852984,2014-05-12,1


**Notice** we also used a UNIQUE constraint as the last part of our CREATE TABLE statement.
This is part of the power of SQL.  Because State and Driver's Licenses numbers should be unique,
we can help limit data input errors by placing a UNIQUE constraint.

#### One-to-many and many-to-one

For a one-to-many/many-to-one relationship (they are inverses of each other),
place the foreign key on the many side of the relationship.

**Example: Customer and Visits**

To find the visits for a customer:

In [12]:
%%sql
SELECT *
FROM visits
WHERE customer_id=1

Done.


created_at,customer_id
2016-06-20,1
2016-07-30,1
2016-04-09,1


To find details on the customer for that visit:

In [13]:
%%sql
SELECT *
FROM customers
WHERE id=1

Done.


id,name,age,city,state
1,john,25,San Francisco,CA


#### Many-to-many

For a many-to-many relationship, create a table that contains two foreign keys,
one to each side of the relationship. This intermediate table is often referred
to as a **JOIN table**.

**Example 1: Customers and Products**

Here is our products table.  It lists all the products in the inventory.

In [14]:
%%sql
DROP TABLE IF EXISTS products;  -- Included so we can run this multiple times without error.

CREATE TABLE products (
  id INTEGER PRIMARY KEY
, name VARCHAR(50)
, price FLOAT
);

INSERT INTO products (name, price) VALUES  -- notice the id will be filled automagically
    ('soccer ball', 20.5),
    ('iPod', 200),
    ('headphones', 50);

SELECT * FROM products

Done.
Done.
3 rows affected.
Done.


id,name,price
1,soccer ball,20.5
2,iPod,200.0
3,headphones,50.0


How do we know which customer purchased which product?  Here is our **JOIN** table.

In [15]:
%%sql
DROP TABLE IF EXISTS purchases;  -- Included so we can run this multiple times without error.

CREATE TABLE purchases (
    customer_id INTEGER REFERENCES customers(id)
,   product_id INTEGER REFERENCES products(id)
,   date TIMESTAMP
,   quantity INTEGER
,   PRIMARY KEY(customer_id, product_id, date));

INSERT INTO purchases VALUES 
    (1, 2, '2016-07-30', 2),
    (2, 3, '2016-06-20', 3),
    (1, 3, '2016-04-09', 1);

SELECT * FROM purchases;

Done.
Done.
3 rows affected.
Done.


customer_id,product_id,date,quantity
1,2,2016-07-30,2
2,3,2016-06-20,3
1,3,2016-04-09,1


**Notice**  In the customers and products tables, there are PRIMARY KEY constraints on the IDs.
This ensures there is only 1 record for each customer and product.  We then used those keys to 
place FOREIGN KEY constraints on the customer_id and product_id in purchases.  This ensures our
data will only contain purchases by customers already in the database for products already in the
database.  Any other scenario indicates our data is not correct.

To find the products purchased by a customer:

In [16]:
%%sql
SELECT products.*
FROM products
JOIN purchases
  ON products.id = purchases.product_id
WHERE purchases.customer_id=1

Done.


id,name,price
2,iPod,200.0
3,headphones,50.0


To find the customers who purchased a product:

In [17]:
%%sql
SELECT customers.*
FROM customers
JOIN purchases
  ON customers.id = purchases.customer_id
WHERE purchases.product_id=3

Done.


id,name,age,city,state
1,john,25,San Francisco,CA
2,becky,30,NYC,NY




## Schema Normalization

When designing a database schema (the tables and columns the database will
contain), we aim to minimize redundancy. This most importantly comes into play
with relations between data. As you saw above, when you want to relate data to
each other, use a simple foreign key - that's the smallest piece of information
that you can keep about another entry.

As an example, each post has an author, which is an entry in the `customers` table.
In a fully normalized schema, the post entry would have a `customer_id` which would
relate the post to a customer. A denormalized way of doing this would be to have a
column in the `posts` table called `author_name`, which would duplicate the
`name` column in the `customers` table. 

Normalization is largely about reducing redundancy. The cost is that some
queries will take longer to run because you will have to look up additional
information in other tables. The benefit is simplicity and a system that is
easier to understand and use. Always start with a fully normalized schema and
then when performance becomes an issue, you can consider selectively
denormalizing.

You can learn more about database normalization from [Wikipedia][wiki-normal].

[wiki-normal]: http://en.wikipedia.org/wiki/Database_normalization

## SQL

Structured Query Language (SQL) is the language used to query relational
databases. All RDBMS use SQL and the syntax and keywords are for the most part
the same across systems, though each system does have some of its own
peculiarities.

SQL is used to interact with RDBMS. That is, it allows you to create tables,
alter tables, insert records, update records, delete records, and query for
records within and across tables.

**We will focus primarily on querying.**

SQL, unlike Python or many other general purpose programming languages, is a
declarative language, meaning the query describes the set of results. Here's an
example of a simple query:

```sql
SELECT name, age
FROM customers
```

This query returns the name and age for every customer in the `customers` table.

```sql
SELECT name, age
FROM customers
WHERE state = 'CA'
```

This query returns the name and age for every customer in the `customers` table who
lives in CA.

## SQL Queries

SQL queries are composed of **clauses**. Each clause begins with a **keyword**.
Every query begins with the `SELECT` clause followed by the `FROM` and
`JOIN` clauses. You then have the ability to apply filtering, aggregation, and
ordering clauses. 

While `SELECT` appears at the top of the query, it is actually very nearly the
last part of the query that is executed. `SELECT` specifies the columns that
should be returned. 

**The most important parts of any SQL query are the `FROM` and `JOIN` clauses.**
These are the first parts of the query to be evaluated, specifying the rows upon
which all the filtering, aggregation, and ordering functions are applied.

We'll briefly cover `JOIN` and variants and then move to a conceptual overview
of the order of execution of a SQL query followed by a detailed example.

### Aggregations

SQL allows you to aggregate your data set based on common keys.  To see the number of visits
from each customer_id we would query:

In [18]:
%%sql
SELECT customer_id, COUNT(*)
FROM visits
GROUP BY customer_id

Done.


customer_id,COUNT(*)
1,3
2,1
3,1


**Notice** The GROUP BY clause tells SQL what common factor we'd like to use to aggregate the data.
The COUNT aggregate function tells SQL how we'd like to aggregate.

### Aggregations Cont.
When we JOIN tables we are essentially creating a new table, so we can use aggregate functions when 
using JOINs.  To get the amount of revenue from each product:

In [19]:
%%sql
SELECT products.name, products.id, SUM(purchases.quantity * products.price) AS revenue
FROM products
JOIN purchases 
  ON products.id=purchases.product_id
GROUP BY products.name, products.id

Done.


name,id,revenue
headphones,3,200.0
iPod,2,400.0


### Sorting

To make the results more readable, we can sort them.  Maybe we want the customers in alphabetical order.

In [20]:
%%sql
SELECT *
FROM customers
ORDER BY NAME

Done.


id,name,age,city,state
2,becky,30,NYC,NY
1,john,25,San Francisco,CA
3,sarah,20,Denver,CO


And if we wanted them in reverse alphabetical order, we'd add the DESC keyword.

In [21]:
%%sql
SELECT *
FROM customers
ORDER BY NAME DESC

Done.


id,name,age,city,state
3,sarah,20,Denver,CO
1,john,25,San Francisco,CA
2,becky,30,NYC,NY


### SQL Order of Operations

SQL does not perform operations "top to bottom".  Rather it executes statements in the following order:

1. **FROM**, **JOIN**: first the product of all tables is formed
2. **WHERE**: the where clause is used to filter rows not satisfying search conditions
3. **GROUP BY** + (**COUNT**, **SUM**, etc): rows are grouped using the columns in the group by clause and the aggregation functions are applied
4. **HAVING**: like the **WHERE** clause, but can be applied after aggregation
5. **SELECT**: the targeted list of columns are evaluated and returned
6. **ORDER BY**: the resulting rows are sorted


### `JOIN` types

The various `JOIN`s specify how to deal with different circumstances regarding
the primary and foreign key matchings. 

`INNER JOIN`s discard any entries that do not have a match between the keys
specified in the `ON` clause. For example, in the above query, any customer who had
not visited the site will NOT be in the result set because a match would not
have been found between the customer's `id` and a visit's `customer_id`.

![inner_join](https://cloud.githubusercontent.com/assets/1425450/9778836/9f669cae-572a-11e5-9c96-98b59a930b7d.png)

A `LEFT OUTER JOIN` keeps all the entries in the left table regardless of
whether a match is found in the right table. In that case, the columns
associated with the right table for that entry will simply be `NULL`. A `RIGHT
OUTER JOIN` is the same except it keeps all the entries in the right table
instead of the left one.

![left_join](https://cloud.githubusercontent.com/assets/1425450/9778839/9f69bbd2-572a-11e5-9b13-7b2c2d7a04fb.png)

![right_join](https://cloud.githubusercontent.com/assets/1425450/9779109/19ace62e-572d-11e5-9868-17a9a7e3440f.png)

In [22]:
%%sql
SELECT c.id, l.number
FROM customers c
LEFT JOIN licenses l
  ON l.customer_id = c.id

Done.


id,number
1,DL19852984
2,None
3,DL19480284


A `FULL OUTER JOIN` will keep the rows of both tables no matter what with `NULL`
values for ones that don't have matches.

![full_outer_join](https://cloud.githubusercontent.com/assets/1425450/9778837/9f66b90a-572a-11e5-9d29-2b6c817cc7ec.png)

# Advanced SQL

To be a SQL wizard, it's important to become very comfortable with SQL joins, nested `SELECTS` and `GROUP BY`s.

Let's say for an online sales company you have the following tables:
```sql
CREATE TABLE CUSTOMERS (
    id INTEGER PRIMARY KEY
,   name VARCHAR(50)
,   age INTEGER
,   city VARCHAR(255)
,   state VARCHAR(2)
    );
    
CREATE TABLE PRODUCTS (
	id INTEGER PRIMARY KEY
,	name VARCHAR(50)
,	price FLOAT
	);
    
CREATE TABLE purchases (
    customer_id INTEGER REFERENCES customers(id)
,    product_id INTEGER REFERENCES products(id)
,    date TIMESTAMP
,    quantity INTEGER );

CREATE TABLE VISITS (
  id INTEGER PRIMARY KEY
,  created_at TIMESTAMP
,  customer_id INTEGER REFERENCES customers(id) );
```

### Inner Join

![inner_join](https://cloud.githubusercontent.com/assets/1425450/9778836/9f669cae-572a-11e5-9c96-98b59a930b7d.png)

If you want to get all the purchases but you want to include the customer name and the product name, you would use a stardard join (this is also called an *inner join*):

```sql
SELECT customers.name AS cust_name, products.name AS prod_name, date, quantity
FROM purchases purchases
JOIN products 
    ON products.id=purchases.product_id
JOIN customers 
    ON customers.id=purchases.customer_id;
```

### Outer Join

Let's say there's a mistake in our database and some products are in the purchase table but not in the products table. With the above inner join query, they would just end up being excluded. An inner join only includes entries which are in both tables.

In [23]:
%%sql -- Table with mistakes

DROP TABLE IF EXISTS purchases_no_key;

CREATE TABLE purchases_no_key (
    customer_id INTEGER REFERENCES customers(id)
,   product_id INTEGER REFERENCES products(id)
,   date TIMESTAMP
,   quantity INTEGER
,   PRIMARY KEY(customer_id, product_id, date));

INSERT INTO purchases_no_key (customer_id, product_id, date, quantity) VALUES
    (1, 2, '2015-07-30', 2)
,   (2, 4, '2015-06-20', 3)
,   (1, 3, '2015-04-09', 1);

Done.
Done.
3 rows affected.


[]

We can ensure all records are included in our result by using an *outer join*.

In [24]:
%%sql
SELECT customers.name AS cust_name, products.name AS prod_name, date, quantity
FROM purchases_no_key purchases
LEFT OUTER JOIN products 
    ON products.id=purchases.product_id
JOIN customers 
    ON customers.id=purchases.customer_id;

Done.


cust_name,prod_name,date,quantity
john,iPod,2015-07-30,2
becky,None,2015-06-20,3
john,headphones,2015-04-09,1


![left_join](https://cloud.githubusercontent.com/assets/1425450/9778839/9f69bbd2-572a-11e5-9b13-7b2c2d7a04fb.png)

The LEFT means that everything from the first table will be included even if there isn't a matching entry in the second table. This will include those missing products, but their names will be `NULL`.

We'll want to isolate those mistakes, we can use the same query as above, but only returns rows where
something in the left table isn't in the joined table.

In [25]:
%%sql
SELECT purchases.*
FROM purchases_no_key purchases
LEFT OUTER JOIN products 
    ON products.id=purchases.product_id
JOIN customers 
    ON customers.id=purchases.customer_id
WHERE products.name IS NULL;

Done.


customer_id,product_id,date,quantity
2,4,2015-06-20,3


![left_join_null](https://cloud.githubusercontent.com/assets/1425450/9779111/1b8b0df4-572d-11e5-922d-55d7b2d36dd2.png)

Even better, we can create the Purchases table with a FOREIGN KEY constraint to ensure that all customer and product ids are in their respective tables.  If you try the below, you will receive an error:

In [26]:
%sql PRAGMA foreign_keys = ON;

Done.


[]

<!--
```sql
INSERT INTO purchases (customer_id, product_id, date, quantity) VALUES
   (2, 4, '2015-06-20', 3);

ERROR:  insert or update on table "purchases" violates foreign key constraint "purchases_product_id_fkey"
DETAIL:  Key (product_id)=(4) is not present in table "products".
```
-->

In [27]:
%%sql
INSERT INTO purchases (customer_id, product_id, date, quantity) VALUES
   (2, 4, '2015-06-20', 3);

IntegrityError: (sqlite3.IntegrityError) FOREIGN KEY constraint failed [SQL: "INSERT INTO purchases (customer_id, product_id, date, quantity) VALUES\n   (2, 4, '2015-06-20', 3);"]

### Except


You can also use `EXCEPT` (MINUS in some versions of SQL):

In [28]:
%%sql
SELECT purchases.product_id
FROM purchases_no_key purchases
EXCEPT
SELECT products.id
FROM products;

Done.


product_id
4


### Getting the maximum

For this we will assume that there's no missing data in our tables.

There is a MAX function which gives you the maximum value, but often you want the entry associated with it. Let's say you want to find the most expensive item. The way to do this is to find the maximum price and then find all the elements whose price is that value.

This requires you to do a nested SELECT statement.

In [29]:
%%sql
SELECT *
FROM products
WHERE price=(
            SELECT MAX(price) 
            FROM products
            ); 

Done.


id,name,price
2,iPod,200.0


Or maybe you want the product which has generated the most profit. Sometimes it's helpful to, instead of having multiple nested selects, creating a temporary table as an intermediary step.

In [30]:
%%sql
DROP TABLE IF EXISTS profits;

CREATE TEMPORARY TABLE profits AS
SELECT name, products.id AS product_id, SUM(purchases.quantity * products.price) AS profit
FROM products
JOIN purchases
    ON products.id=purchases.product_id
GROUP BY products.name, products.id;

SELECT name, product_id
FROM profits
WHERE profit=(SELECT MAX(profit) FROM profits);

Done.
Done.
Done.


name,product_id
iPod,2


You can also do this using a WITH clause:

In [31]:
%%sql
WITH p AS (
SELECT products.name, products.id, SUM(purchases.quantity * products.price) AS profit
FROM products
JOIN  purchases
    ON products.id=purchases.product_id
GROUP BY products.name, products.id)

SELECT name, id
FROM p
WHERE profit=(SELECT MAX(profit) FROM p);

Done.


name,id
iPod,2


<!--
### Windowing Functions

If all of this made sense so far and you are ready for more, checkout [windowing functions](http://www.postgresql.org/docs/9.1/static/tutorial-window.html)

We can get the same result as above using the RANK window function.

```sql
WITH p AS (
SELECT products.name, products.id, SUM(purchases.quantity * products.price) AS profit
FROM products
JOIN  purchases
    ON products.id=purchases.product_id
GROUP BY products.name, products.id)

SELECT name, id, profit
FROM (  SELECT name, id, profit, rank() OVER (ORDER BY profit DESC)
        FROM p) ranked_profit
    WHERE rank=1
```
-->

## Keyword Cheatsheet

* Return values
  * SELECT
  * DISTINCT
* Tables and rows
  * FROM
  * JOIN
    * INNER
    * LEFT, RIGHT
    * FULL
* Filtering
  * WHERE
  * =, !=, >, <, >=, <=
  * AND, OR
  * IN, NOT IN
  * LIKE
  * IS NULL, IS NOT NULL
  * LIMIT
  * BETWEEN
* Aggregating
  * GROUP BY
  * COUNT
  * MAX, MIN
  * SUM
  * AVG
  * HAVING
* Ordering
  * ORDER BY
* Aliasing
  * AS


## Additional Resources

* [SQLite Documentation](https://www.sqlite.org/docs.html)
* [PostgreSQL Documentation](http://www.postgresql.org/docs/9.2/interactive/index.html)
* [SQL for Data Science](http://bensresearch.com/downloads/SQL.pdf)

**Database theory resources**

* [ACID](http://en.wikipedia.org/wiki/ACID)
* [CAP Theorem](http://en.wikipedia.org/wiki/CAP_theorem)
* [Relational Algebra](http://en.wikipedia.org/wiki/Relational_algebra)
* [Set Theory](http://en.wikipedia.org/wiki/Set_theory)

## Lab Exerciese

1. Complete chapters 1, 3, 5 and 6 in this [online SQL tutorial](http://sqlzoo.net/wiki/SQL_Tutorial). The harder questions are optional.  

2. Explore the Titanic database.
    1. Download `titanic.db` from [https://s3-us-west-2.amazonaws.com/dsci/6002/data/titanic.db](https://s3-us-west-2.amazonaws.com/dsci/6002/data/titanic.db)
    2. Open a new terminal and navigate to where you put `titantic.db`
    3. Connect to the database by either:
        1. Using the bash command `sqlite3 titanic.db` on the command line, or
        2. Using the Jupyter magic command: `%sql sqlite:///titanic.db`
    4. Now we are in the database, we can check out the names of the tables in the database by entering the following:  
       `SELECT name FROM sqlite_master WHERE type = "table"`  
    You should see the following output, otherwise ask for help:
  ```
  name
  info
  ```
    5. Answer the following 13 (graded) questions:

**Questions: [GRADED]**

1. Select the first 3 rows in each of the tables (2 tables). State which column contains the primary key.  
    __HINT: .schema can give info on the database in sqlite__ 
2. How many of the passengers are male/female.

3. The percentage of female onboard.

4. Express the ratio of `male:female` onboard.

5. Mean age of passengers

6. Max age of passengers

7. Variance of age of passengers (**SQLite does not have built-in, but most other SQL have**)
    _**HINT: Make a table of `age,average_age` first**_

8. Number of passenger in each `Pclass`

9. Mean `Fare` for each `Pclass`

10. The number of passenger that does not have a name in the `info` table
    _**HINT: Do a `LEFT JOIN` and `INNER JOIN` between `info` and `name` and compare the count**_

11. Repeat `Ex10.` using one `LEFT JOIN` and the `IS NULL` function

12. Find the number of people with `Mr.` in their name

13. Compute the survival rate for male and female.